In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

In [2]:
# Load data
dataset = pd.read_csv("jigsaw-toxic-comment-train.csv")


In [ ]:
dataset.shape

In [4]:
g_classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate','neutral']

In [5]:
classes = ['Safe','Dangerous']

In [6]:
dataset['neutral'] = ((dataset[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) == 0).astype(int))

In [ ]:
class_counts = dataset[g_classes].sum()
print(class_counts)

In [8]:
# Assuming `df` is your DataFrame
def classify(row):
    
    if row['severe_toxic'] == 1 or row['threat'] == 1 or row['insult'] == 1 or row['identity_hate'] == 1 or row['toxic'] == 1 or row['obscene'] == 1:
        return 1

    elif row['neutral'] == 1:
        return 0
    
    return 0

dataset['Dangerous'] = dataset.apply(lambda row: pd.Series(classify(row)), axis=1)


In [9]:
dataset.drop(columns=g_classes,axis=1,inplace=True)

In [ ]:
dataset.head(20)

In [11]:
# Separate the data by class
df = pd.DataFrame()

class_0 = dataset[dataset['Dangerous'] == 0]
class_1 = dataset[dataset['Dangerous'] == 1]

# Randomly sample 10,000 cases from each class
sample_class_0 = class_0.sample(n=10000, random_state=42)
sample_class_1 = class_1.sample(n=10000, random_state=42)

# Combine the samples into a new DataFrame
df = pd.concat([sample_class_0, sample_class_1])

# Optionally shuffle the data
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
class_counts = df['Dangerous'].sum()
print(class_counts)

In [13]:
import re

def preprocess_text(text):
    # Basic text preprocessing
    text = text.lower()  # Lowercase text
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

# Apply preprocessing on your dataset
df['comment_text'] = df['comment_text'].apply(preprocess_text)

In [ ]:
df.head()


In [15]:
df.to_csv('preprocessed_data')

In [16]:
df = pd.read_csv('preprocessed_data')


In [17]:
texts = ['comment_text']
label = ['Dangerous']

In [18]:
from sklearn.model_selection import train_test_split
X = pd.DataFrame(df.drop(columns=label))
y = pd.DataFrame(df.drop(columns=texts))
X_train,X_eval_test,y_train,y_eval_test = train_test_split(X,y,test_size = 0.5,random_state=42)
X_val,X_test,y_val,y_test = train_test_split(X_eval_test,y_eval_test,test_size=0.5,random_state=42)

In [ ]:
print((X_train).shape)
print((X_val).shape)
print((X_test).shape)


In [20]:
# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [21]:
# Tokenization function using Hugging Face tokenizer
def encode_texts(texts, tokenizer, maxlen=512):
    return tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=maxlen,
        return_tensors='pt'
    )


In [22]:
# Encode datasets
x_train = encode_texts(X_train.comment_text.astype(str), tokenizer, maxlen=192)
x_valid = encode_texts(X_val.comment_text.astype(str), tokenizer, maxlen=192)
x_test = encode_texts(X_test.comment_text.astype(str), tokenizer, maxlen=192)

In [ ]:
y.columns

In [24]:
y_train = torch.tensor(y_train[label].values)
y_valid = torch.tensor(y_val[label].values)
y_test = torch.tensor(y_test[label].values)

In [ ]:
x_test

In [26]:
# Custom dataset class
class ToxicCommentsDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [27]:
# Dataset and Dataloader
train_dataset = ToxicCommentsDataset(x_train, y_train)
valid_dataset = ToxicCommentsDataset(x_valid, y_valid)

train_loader = DataLoader(train_dataset, batch_size=16)  # Try reducing batch size
valid_loader = DataLoader(valid_dataset, batch_size=16)


In [ ]:
# Load pre-trained BERT model
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-multilingual-cased',
    num_labels=1
)

In [ ]:
# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=1e-4)
total_steps = len(train_loader) * 3  
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0, 
                                            num_training_steps=total_steps)

# Loss function
loss_fn = torch.nn.BCEWithLogitsLoss()

In [30]:
# Training loop
def train_epoch(model, dataloader, optimizer, device, scheduler, loss_fn):
    model.train()  # Set the model to training mode
    total_loss = 0  # Track total loss for the epoch
    correct_predictions = 0  # Track correct predictions (if needed for accuracy)
    
    # Iterate through the dataloader batches
    for batch in tqdm(dataloader):
        optimizer.zero_grad()  # Zero the gradients before each batch

        # Move inputs and labels to the GPU (or CPU if no GPU)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].squeeze(1).float().to(device)  # Ensure labels are float type
        
        # Forward pass through the model
        outputs = model(input_ids, attention_mask=attention_mask)
        
        # Compute the loss between model predictions and actual labels
        loss = loss_fn(outputs.logits.squeeze(1), labels)  # Squeeze outputs to match labels
        total_loss += loss.item()  # Accumulate total loss
        
        # Backward pass and optimization step
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Compute average loss over the epoch
    avg_loss = total_loss / len(dataloader)
    
    return avg_loss

In [31]:
def eval_model(model, dataloader, device, loss_fn):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for batch in dataloader:
            # Move inputs and labels to the GPU (or CPU if no GPU)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].squeeze(1).float().to(device)

            print(f'Input IDs shape: {input_ids.shape}, Attention Mask shape: {attention_mask.shape}, Labels shape: {labels.shape}')  # Debugging shapes

            # Forward pass through the model
            outputs = model(input_ids, attention_mask=attention_mask)

            print(f'Output shape: {outputs.logits.shape}')  # Debugging output shape

            # Compute the loss
            loss = loss_fn(outputs.logits.squeeze(1), labels)  # Squeeze outputs to match labels
            total_loss += loss.item()

            # Apply sigmoid to logits to get probabilities and threshold to get binary predictions
            preds = torch.sigmoid(outputs.logits.squeeze(1))  # Squeeze preds to match labels
            preds = (preds > 0.75).float()  # Convert probabilities to binary (0 or 1)

            # Calculate correct predictions
            correct_predictions += (preds == labels).sum().item()
            total_samples += labels.size(0)

    avg_loss = total_loss / len(dataloader)  # Average loss over the dataloader
    accuracy = correct_predictions / total_samples if total_samples > 0 else 0  # Compute accuracy
    return avg_loss, accuracy


In [32]:
device = 'cpu'

In [ ]:
# Assuming your training loop is already defined as you provided
epochs = 10
best_val_loss = float('inf')  # Initialize best validation loss to infinity
best_val_accuracy = 0.0  # Initialize best validation accuracy to 0

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    
    # Training phase
    train_loss = train_epoch(model, train_loader, optimizer, device, scheduler, loss_fn)
    
    # Validation phase
    val_loss, val_accuracy = eval_model(model, valid_loader, device, loss_fn)  
    
    print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}")

    # Save model if validation loss has improved
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model_loss1.pth')  
        print(f"Best model saved based on Validation Loss: {val_loss:.4f}")

    # Save model if validation accuracy has improved
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_model_accuracy1.pth')
        print(f"Best model saved based on Validation Accuracy: {val_accuracy:.4f}")


# TESTING

In [57]:
test_dataset = ToxicCommentsDataset(x_test, y_test)

test_loader = DataLoader(test_dataset, batch_size=16)

In [58]:
def test_model(model, dataloader, device):
    model.eval()  # Set model to evaluation mode
    correct_predictions = 0
    total_samples = 0
    predictions = []
    x = 0

    with torch.no_grad():  # Disable gradient calculation for testing
        for batch in dataloader:
            # Move inputs to the device (GPU/CPU)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Forward pass through the model
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.sigmoid(outputs.logits.squeeze(1))  # Apply sigmoid to logits
            preds = (preds > 0.75).float()  # Convert probabilities to binary (0 or 1)

            predictions.extend(preds.cpu().numpy())  # Store predictions

            if 'labels' in batch:  # If labels are available for evaluation
                labels = batch['labels'].squeeze(1).float().to(device)
                correct_predictions += (preds == labels).sum().item()
                total_samples += labels.size(0)
            print(x)
            x=x+1

    # Compute accuracy if labels were provided
    accuracy = correct_predictions / total_samples if total_samples > 0 else None
    
    return predictions, accuracy


In [ ]:
# Load the best model weights (based on loss or accuracy)
model.load_state_dict(torch.load('best_model_accuracy.pth'))  # or 'best_model_loss.pth'

In [ ]:
# Test the model on the test dataset
predictions, test_accuracy = test_model(model, test_loader, device)

if test_accuracy is not None:
    print(f"Test Accuracy: {test_accuracy:.2f}")
else:
    print("Test completed. Predictions generated.")


## CUSTOM TEST DATA

In [ ]:
# Load pre-trained BERT model
device = 'cpu'
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-multilingual-cased',
    num_labels=1
)
model.load_state_dict(torch.load('best_model_accuracy_final.pth'))  
model.to(device)  

In [3]:
def predict_custom_data(model, input_ids, attention_mask, device):
    model.eval()  # Set model to evaluation mode

    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(input_ids, attention_mask=attention_mask)
        preds = torch.sigmoid(outputs.logits.squeeze(1))  # Apply sigmoid to get probabilities
        preds = (preds > 0.5).float()  # Convert probabilities to binary (0 or 1)

    return preds.cpu().numpy()  # Return predictions as a numpy array


In [4]:
# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [5]:
custom_texts = ["please dont hurt me"] 

max_length = 128  # 

encoded_inputs = tokenizer(
    custom_texts,
    truncation=True,
    padding=True,
    max_length=max_length,
    return_tensors='pt'  
)

input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)


In [ ]:
# Predict on custom data
predictions = predict_custom_data(model, input_ids, attention_mask, device)

# Print the predictions
for i, text in enumerate(custom_texts):
    print(f"Text: {text} -> Prediction: {'Dangerous' if predictions[i] == 1 else 'Safe'}")
